In [1]:
import cv2
import numpy as np
from random import randint
from copy import deepcopy
cv2.destroyAllWindows()

In [2]:
#Fungi tambahan
def inter(x, xMin, xMax, yMin, yMax):
    return yMin + x * (yMax-yMin)/(xMax-xMin)

def attract(pos1,pos2, k):
    #Hitung gaya tarik antar 1 yang berada di pos1 dengan 2 yang berada di pos2
    dist = np.linalg.norm(pos1 - pos2)
    return -dist**2/k 

def repul(pos1, pos2, k):
    #Hitung gaya dorong antar 1 yang berada di pos1 dengan 2 yang berada di pos2
    dist = np.linalg.norm(pos1 - pos2)
    return (k**2)/dist

In [20]:
#Class Node
class Node:
    #Attr
    idNode = 0
    pos = np.array((0,0))
    delta = np.array((0,0))
    tetangga = []
    #Method
    def __init__(self, _idNode, _pos):
        self.idNode = _idNode
        self.pos = _pos
        self.delta = np.array((0,0))
        self.tetangga = []
    def getId(self):
        return self.idNode
    def setId(self, _id):
        self.idNode = _id
    def setPos(self , _pos):
        self.pos = _pos
    def getPos(self):
        return self.pos
    def setDelta(self, _delta):
        self.delta = _delta
    def getDelta(self):
        return self.delta
    def update(self):
        self.pos += self.delta
        self.delta = np.array((0,0))
        padx = 50
        pady = 50
        self.pos[0] = max(0+padx, self.pos[0])
        self.pos[0] = min(800-padx, self.pos[0])
        self.pos[1] = max(0+pady, self.pos[1])
        self.pos[1] = min(800-pady, self.pos[1])
    def addTetangga(self, t):
        self.tetangga.append(t)
    def getTetangga(self):
        return self.tetangga
        
#Class Graph
class Graph:
    #Attr
    listOfNode = []
    k = 1 #konstanta
    #Method
    def __init__(self, n):
        #Init dengan n node
        self.listOfNode = []
        for i in range(n):
            j = int(np.ceil(np.sqrt(n)))
            x = inter(i % j, 0, j-1, 50, 750)
            y = inter(int(i / j), 0, j-1, 50, 750)
            self.listOfNode.append(Node(i, np.array((x,y))))
        #self.printStatus()
        #Hitung konstanta
        #self.k = 100
        self.k = np.sqrt(800*800/n)
    def getListOfNode(self):
        return self.listOfNode
    def addEdge(self, n1, n2):
        #n1, n2 = nomor dari node
        self.listOfNode[n1].addTetangga(n2)
        self.listOfNode[n2].addTetangga(n1)
 
    def updateFD(self):
        #Versi tanpa barnes-hut (divide & conquer)
        dt = 0.001
        maxIter = 1
        for it in range(maxIter):
            #Untuk setiap node yang tetanggaan , beri gaya tarik
            for i, n in enumerate(self.listOfNode):
                for t in n.getTetangga():
                    att = attract(n.getPos(), self.listOfNode[t].getPos(), self.k)
                    arah = -(self.listOfNode[t].getPos()-n.getPos())
                    arah = arah/np.linalg.norm(arah)
#                     print('Attr : ',i, ' ', t, ' ', n.getDelta()+arah*att*dt)
                    n.setDelta(n.getDelta()+arah * att * dt)
            #Untuk setiap node dengan lainnya, beri gaya repulsif
            for i, n in enumerate(self.listOfNode):
                for j, n2 in enumerate(self.listOfNode):
                    if(i != j):
                        rep = repul(n.getPos(), n2.getPos(), self.k)
                        arah = -(n2.getPos()-n.getPos())
                        arah = arah/np.linalg.norm(arah)
                        #print('Repel : ',i, ' ', j, ' ', n.getDelta()+arah*rep*dt)
                        n.setDelta(n.getDelta()+arah * rep * dt)
                forceTengahLayar = (n.pos - np.array((400,400)))
                jarakTengahLayar = np.linalg.norm(forceTengahLayar)
                forceTengahLayar /= jarakTengahLayar
                forceTengahLayar *= attract(n.pos, np.array((400,400)), 10*self.k)
                n.setDelta(n.getDelta()+forceTengahLayar*dt*100)
            #Update posisi semuanya
            for n in self.listOfNode:
                #print(n.getPos(), ' ', n.getDelta())
                n.update()
#             print('Iter : ', it)
#             self.printStatus()
#             print('=====================')
    def printStatus(self):
        for n in self.listOfNode:
            print(n.getId(),' : ')
            print('  Pos   : ',n.getPos())
            print('  Delta : ',n.getDelta())
            print('  Tetangga : ')
            for t in n.getTetangga():
                print('    ', t)
            print('===============')

    def printGraph(self):
        #init awal
        while(True):
            img = np.zeros((800,800,3), dtype='uint8')
            for i, n in enumerate(self.listOfNode):
                center = (int(n.getPos()[0]), int(n.getPos()[1]))
                for t in n.getTetangga():
                    centerT = (int(self.listOfNode[t].getPos()[0]), int(self.listOfNode[t].getPos()[1]))
                    img = cv2.line(img, center, centerT, (0, 255, 0), 2)
                warna = (255, 0, 255)
                img = cv2.circle(img, center, 3, warna, 3)
            cv2.imshow('graph', img)
            self.updateFD()
            #print(self.listOfNode[0].getPos())
            key = cv2.waitKey(1)
            if(key == ord('q')):
                break
        cv2.destroyAllWindows()

In [43]:
class Body:

    def __init__(self, _id, _pos, _vel):
        self.id = _id
        self.pos = _pos
        self.vel = _vel
        self.force = np.array((0,0))
        self.tetangga = []
    def update(self, dt):
        self.vel = np.add(self.vel, (self.force * dt))
#         batasX = 50
#         batasY = 50
#         self.vel[0] = max(self.vel[0], -batasX)
#         self.vel[0] = min(self.vel[0], batasX)
#         self.vel[1] = max(self.vel[1], -batasY)
#         self.vel[1] = min(self.vel[1], batasY)
        self.pos = np.add(self.pos, (self.vel * dt))
        padx = 50
        pady = 50
        self.pos[0] = max(0+padx, self.pos[0])
        self.pos[0] = min(800-padx, self.pos[0])
        self.pos[1] = max(0+pady, self.pos[1])
        self.pos[1] = min(800-pady, self.pos[1])
    def distance(self, b):
        delta = self.pos - b.getPos()
        return np.linalg.norm(delta)
    def setForce(self, _force):
        self.force = _force
    def addTetangga(self, idT):
        if(not(idT in self.tetangga)):
            self.tetangga.append(idT)
    def getPos(self):
        return self.pos
    def getTetangga(self):
        return self.tetangga
    
class BarnesHut:

    def __init__(self, _level, _center, _p, _l, _trueCenter):
        self.level = _level
        self.center = deepcopy(_center)
        self.trueCenter = deepcopy(_trueCenter)
        self.kuadran = [None, None, None, None]
        self.totalMass = 0
        self.panjang = _p
        self.lebar = _l
        self.isi = []
        self.isiId = []
        self.body = None
        self.numOfBody = 0
    def getImg(self):
        temp = np.zeros((800,800,3), dtype = 'uint8')
        horP1 = tuple((self.trueCenter - np.array((self.panjang/2, 0))).astype(int))
        horP2 = tuple((self.trueCenter + np.array((self.panjang/2, 0))).astype(int))
        verP1 = tuple((self.trueCenter - np.array((0, self.lebar/2))).astype(int))
        verP2 = tuple((self.trueCenter + np.array((0, self.lebar/2))).astype(int))
        if(any(self.kuadran)):
            temp = cv2.line(temp, horP1, horP2, (255,255,255), 2)
            temp = cv2.line(temp, verP1, verP2, (255,255,255), 2)
        if(self.kuadran[0] != None):
            temp = cv2.bitwise_or(temp, self.kuadran[0].getImg())
        if(self.kuadran[1] != None):
            temp = cv2.bitwise_or(temp, self.kuadran[1].getImg())
        if(self.kuadran[2] != None):
            temp = cv2.bitwise_or(temp, self.kuadran[2].getImg())
        if(self.kuadran[3] != None):
            temp = cv2.bitwise_or(temp, self.kuadran[3].getImg())
        return temp

    def getKuadran(self, pos):
        #Menentukan pos ada di kuadran mana
        if(pos[0] >= self.trueCenter[0]):
            if(pos[1] >= self.trueCenter[1]):
                return 3
            else:
                return 0
        else:
            if(pos[1] >= self.trueCenter[1]):
                return 2
            else:
                return 1
      
    def allocKuadran(self, noKuadran):
        if(noKuadran == 0):
            temp = np.array((self.panjang/4, -self.lebar/4))
            self.kuadran[0] = BarnesHut(self.level+1,self.center+temp, 2*temp[0], -2*temp[1], self.trueCenter+temp)
        elif(noKuadran == 1):
            temp = np.array((-self.panjang/4, -self.lebar/4))
            self.kuadran[1] = BarnesHut(self.level+1,self.center+temp, -2*temp[0], -2*temp[1], self.trueCenter+temp)
        elif(noKuadran == 2):
            temp = np.array((-self.panjang/4, self.lebar/4))
            self.kuadran[2] = BarnesHut(self.level+1,self.center+temp, -2*temp[0], 2*temp[1], self.trueCenter+temp)
        elif(noKuadran == 3):
            temp = np.array((self.panjang/4, self.lebar/4))
            self.kuadran[3] = BarnesHut(self.level+1,self.center+temp, 2*temp[0], 2*temp[1], self.trueCenter+temp)
  
    def addBody(self, b):
        if(not(b.id in self.isiId)):
            self.isiId.append(b.id)
        if(self.level > 10):
            self.isi.append(b)
            self.center = (self.center * self.totalMass + b.pos)/(self.totalMass+1)
            self.totalMass += 1
        else:
            #print('Level ', self.level)
            if(self.numOfBody > 1):
                noKuad = self.getKuadran(b.pos)
                if(self.kuadran[noKuad] == None):
                    self.allocKuadran(noKuad)
                #print('Masuk ke ', noKuad)
                self.kuadran[noKuad].addBody(b)
                self.center = (self.center * self.totalMass + b.pos)/(self.totalMass+1)
                self.totalMass += 1
            elif(self.numOfBody == 1):
                #print('Pecah')
                noKuad = self.getKuadran(self.isi[0].pos)
                self.allocKuadran(noKuad)
                self.kuadran[noKuad].addBody(self.isi[0])
                self.isi = []
                noKuad2 = self.getKuadran(b.pos)
                if(noKuad != noKuad2):
                    self.allocKuadran(noKuad2)
                self.kuadran[noKuad2].addBody(b)
                self.center = (self.center * self.totalMass + b.pos)/(self.totalMass+1)
                self.totalMass += 1
            elif(self.numOfBody == 0 and len(self.isi) == 0):
                #print('Wah kosong')
                self.isi.append(b)
                self.totalMass = 1
                self.center = deepcopy(b.pos)
        self.numOfBody += 1

    def getForceOn(self, b, tetha, k):
        #Return force di body b dengan parameter tetha
        d = np.array((self.panjang, self.lebar))
        d = np.linalg.norm(d)
        force = (b.pos - self.center)
        jarak = np.linalg.norm(force)
        force /= jarak
        
        forceTengahLayar = (b.pos - np.array((400,400)))
        jarakTengahLayar = np.linalg.norm(forceTengahLayar)
        forceTengahLayar /= jarakTengahLayar
        forceTengahLayar *= attract(b.pos, np.array((400,400)), 10*k)
        if(self.numOfBody == 1):
            temp = 0
            if(self.isi[0].id != b.id):
                if(self.isi[0].id in b.tetangga):
                    temp += attract(self.isi[0].pos,b.pos, k)
                temp += repul(self.isi[0].pos,b.pos, k)
            return force * temp + forceTengahLayar
        else:
            if(d/jarak < tetha):
                temp = repul(self.center, b.pos, k)
                if(b.id in self.isiId):
                    temp += attract(self.center, b.pos, k)
                return force * temp + forceTengahLayar
            else:
                force = np.array((0,0))
                for kuad in self.kuadran:
                    temp = 0
                    if(kuad != None):
                        temp = kuad.getForceOn(b, tetha, k)
                        #print(temp)
                        if(np.isnan(temp[0]) or np.isnan(temp[1])):
                            temp = np.array((0,0))
                        force = np.add(force, temp)
                return force + forceTengahLayar
            
class GraphBarnes:
    #Attr
    listOfNode = []
    k = 1 #konstanta
    #Method
    def __init__(self, n):
        #Init dengan n node
        self.listOfNode = []
        self.barnes = BarnesHut(0, np.array((400,400)),800, 800, np.array((400,400)))
        for i in range(n):
            j = int(np.ceil(np.sqrt(n)))
            x = inter(i % j, 0, j-1, 50, 750)
            y = inter(int(i / j), 0, j-1, 50, 750)
            pos = np.array((x, y))
            vel = np.array((0,0))
            self.listOfNode.append(Body(i, pos, vel))
            self.barnes.addBody(self.listOfNode[i])
        #self.printStatus()
        #Hitung konstanta
        #self.k = 50
        self.k = np.sqrt(800*800/n)
    def getListOfNode(self):
        return self.listOfNode
    def addEdge(self, n1, n2):
        #n1, n2 = nomor dari node
        self.listOfNode[n1].addTetangga(n2)
        self.listOfNode[n2].addTetangga(n1)

    def updateFD(self):
        for i in range(len(self.listOfNode)):
            force = (1/20)*self.barnes.getForceOn(self.listOfNode[i], 0.5, self.k)
            self.listOfNode[i].setForce(force)
        self.barnes = BarnesHut(0, np.array((400,400)),800, 800, np.array((400,400)))
        for i in range(len(self.listOfNode)):
            #for j in range(10):
            self.listOfNode[i].update(0.1)
            self.listOfNode[i].setForce(np.array((0,0)))
            self.barnes.addBody(self.listOfNode[i])
    def printStatus(self):
        for n in self.listOfNode:
            print(n.getId(),' : ')
            print('  Pos   : ',n.getPos())
            print('  Delta : ',n.getDelta())
            print('  Tetangga : ')
            for t in n.getTetangga():
                print('    ', t)
            print('===============')

    def printGraph(self):
        #init awal
        while(True):
            img = np.zeros((800,800,3), dtype='uint8')
            #img = cv2.bitwise_or(img, self.barnes.getImg())
            for i, n in enumerate(self.listOfNode):
                center = (int(n.getPos()[0]), int(n.getPos()[1]))
                for t in n.getTetangga():
                    centerT = (int(self.listOfNode[t].getPos()[0]), int(self.listOfNode[t].getPos()[1]))
                    img = cv2.line(img, center, centerT, (0, 255, 0), 2)
                warna = (255, 0, 255)
                img = cv2.circle(img, center, 3, warna, 3)
            cv2.imshow('graph', img)
            self.updateFD()
            #print(self.listOfNode[0].getPos())
            key = cv2.waitKey(1)
            if(key == ord('q')):
                break
        cv2.destroyAllWindows()
    

In [18]:
g = GraphBarnes(10)
g.addEdge(1, 2)
g.addEdge(3, 2)
g.addEdge(0, 3)
g.addEdge(5, 8)
g.addEdge(3, 9)
g.addEdge(2, 5)
g.addEdge(1, 7)
g.printGraph()

/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
cv2.destroyAllWindows()

In [48]:
g = GraphBarnes(2000)
for i in range(2000-1):
        g.addEdge(i, 2000 - i - 1)

g.printGraph()

/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:139: RuntimeWarning: invalid value encountered in true_divide
/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:153: RuntimeWarning: divide by zero encountered in double_scalars
/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:143: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
g = Graph(2000)
for i in range(2000-1):
        g.addEdge(i, 2000 - i - 1)

g.printGraph()

/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:89: RuntimeWarning: invalid value encountered in true_divide
